In [ ]:
import numpy as np
import tensorflow as tf
import tflearn
import random

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("Processing the intents....")
with open('intents.json') as json_data:
    intents = json.load(json_data)


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the INtents to convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our word list
        words.extend(w)
        # add to documents in corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            

In [ ]:
print("Stemming, lowering and Removing Duplicates.....")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)


In [ ]:
print("Creating the data for our model.......")
training = []
output = []
print("Creating an empty list for output.....")
output_empty = [0] * len(classes)

print("Creating Training set, Bag of words fro our model......")
for doc in document:
    bag=[]
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.appenf([bag, output_row])

In [ ]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and test Lists.........")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for our Chatbot to be Contextual.....")
print("Resetting graph data.....")
tf.reset_default_graph()

In [ ]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regeression(net)
print("Training......")


In [ ]:
model = tflearn.DNN(net, tensorborad_dir="tflearn_logs")

In [ ]:
print("Training the Model.....")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Retrieving the model......")
model.save('model.tflearn')

In [ ]:
print("Pickle is also saved....")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [ ]:
print("Loading Pickle........")
data = pickle.load( open( "training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the model.....")
# Load our saved model
model.load('./model.tflearn')

In [ ]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentences(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
                
            results.pop(0)

In [ ]:
while True:
    input_data = input("you- ")
    answer = response(input_data)
    answer